In [1]:
import datasets

/opt/homebrew/Caskroom/miniconda/base/envs/AIP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
track = "GSU24AI03-SU24AI21/tracked-url-video"
download = "GSU24AI03-SU24AI21/downloaded-vietnamese-video"
speaker = "GSU24AI03-SU24AI21/detected-speaker-clip"
crop = "GSU24AI03-SU24AI21/cropped-mouth-clip"
vndetect = "GSU24AI03-SU24AI21/detected-vietnamese-clip"
trans = "GSU24AI03-SU24AI21/transcribed-vietnamese-audio"
vnav = "GSU24AI03-SU24AI21/vietnamese-av"

In [11]:
datasets.get_dataset_config_names(vnav, trust_remote_code=True)

['all', 'batch_99999']

In [36]:
%cd home/vietnamese-av-asr/

/Users/minhnguyen/home/vietnamese-av-asr


In [174]:
!python src/data/tasks/process.py --task download --output-dir data/processed/ --channel-names batch_99999 --overwrite --upload-to-hub

torchvision is not available - cannot save figures
Initialize executor for download task...
Processing channels:   0%|                           | 0/1 [00:00<?, ?channel/s]
-------------------- Processing batch_99999 --------------------
Loading dataset...
Processing data...

Map: 100%|█████████████████████████████████| 1/1 [00:06<00:00,  6.85s/ examples]
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.

Checking number of samples...
Expected 1 in data/processed/downloaded-vietnamese-video/video/batch_99999, but got 1
	Number of samples lost: 0
	Number of output samples: 1
Saving metada...
Uploading to hub...

batch_99999.zip:   0%|                              | 0.00/9.13M [00:00<?, ?B/s]
batch_99999.zip:   0%|                     | 16.4k/9.13M [00:00<01:49, 83.1kB/s]
batch_99999.zip:   2%|▍                      | 197k/9.13M [00:00<00:11, 745kB/s]
batch_99999.zip:  11%|██▍                   | 999k/9.13M [00:00<00:02, 3.27MB/s]
batch_99999.zip:  27%|█████▋          

In [181]:
!python src/data/tasks/process.py --task asd --output-dir data/processed/ --channel-names batch_99999 --overwrite --upload-to-hub

torchvision is not available - cannot save figures
Initialize executor for asd task...
Processing channels:   0%|                           | 0/1 [00:00<?, ?channel/s]
-------------------- Processing batch_99999 --------------------
Loading dataset...
Processing data...

Map:   0%|                                         | 0/1 [00:00<?, ? examples/s]Downloading network results...


(…)work_results@batch_99999@5tTiD5rtVwY.zip:   0%|   | 0.00/309M [00:00<?, ?B/s]

(…)work_results@batch_99999@5tTiD5rtVwY.zip:   3%| | 10.5M/309M [00:01<00:42, 7.

(…)work_results@batch_99999@5tTiD5rtVwY.zip:   7%| | 21.0M/309M [00:03<00:43, 6.

(…)work_results@batch_99999@5tTiD5rtVwY.zip:  10%| | 31.5M/309M [00:04<00:41, 6.

(…)work_results@batch_99999@5tTiD5rtVwY.zip:  14%|▏| 41.9M/309M [00:06<00:38, 6.

(…)work_results@batch_99999@5tTiD5rtVwY.zip:  17%|▏| 52.4M/309M [00:07<00:36, 7.

(…)work_results@batch_99999@5tTiD5rtVwY.zip:  20%|▏| 62.9M/309M [00:09<00:38, 6.

(…)work_results@batch_99999@5tTiD5rtVwY.z

In [193]:
!python src/data/tasks/process.py --task crop --output-dir data/processed/ --channel-names batch_99999 --overwrite --upload-to-hub

torchvision is not available - cannot save figures
Initialize executor for crop task...
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Processing channels:   0%|                           | 0/1 [00:00<?, ?channel/s]
-------------------- Processing batch_99999 --------------------
Loading dataset...
Processing data...
Parameter 'function'=<bound method Cropper.process of <src.data.processors.cropper.Cropper object at 0x312017040>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.

                                                                                
Moviepy - B

In [187]:
!python src/data/tasks/process.py --task vndetect --output-dir data/processed/ --channel-names batch_99999 --overwrite --upload-to-hub

torchvision is not available - cannot save figures
Initialize executor for vndetect task...
Processing channels:   0%|                           | 0/1 [00:00<?, ?channel/s]
-------------------- Processing batch_99999 --------------------
Loading dataset...
Processing data...

Map: 100%|█████████████████████████████████| 9/9 [00:00<00:00, 19.60 examples/s]

Checking number of samples...
Expected 9 in data/processed/detected-vietnamese-clip/visual/batch_99999, but got 9
Expected 9 in data/processed/detected-vietnamese-clip/audio/batch_99999, but got 9
	Number of samples lost: 0
	Number of output samples: 9
Saving metada...
Uploading to hub...

batch_99999.zip:   0%|                               | 0.00/131k [00:00<?, ?B/s]
batch_99999.zip: 100%|███████████████████████| 131k/131k [00:02<00:00, 63.9kB/s]

batch_99999.zip: 100%|████████████████████████| 768k/768k [00:01<00:00, 512kB/s]

batch_99999.parquet: 100%|█████████████████| 4.94k/4.94k [00:00<00:00, 9.72kB/s]

-----------------------

In [199]:
!python src/data/tasks/process.py --task transcribe --output-dir data/processed/ --channel-names batch_99999 --overwrite --upload-to-hub

torchvision is not available - cannot save figures
Initialize executor for transcribe task...
Fetching 4 files: 100%|████████████████████████| 4/4 [00:00<00:00, 48770.98it/s]
Only 0 unigrams passed as vocabulary. Is this small or artificial data?
Processing channels:   0%|                           | 0/1 [00:00<?, ?channel/s]
-------------------- Processing batch_99999 --------------------
Loading dataset...
Processing data...
Parameter 'function'=<bound method Transcriber.process of <src.data.processors.transcriber.Transcriber object at 0x314db2ef0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing fail